In [160]:
%load_ext autoreload
%autoreload 2

from spark.utils import get_channel_names, channel_wise_boss, boss_transform_data
import spark.params as params
from spark.preprocessor import load_q_data, load_timeseries_data, train_test_split_ids, Preprocess_Q
from spark.preprocessor import preprocessor

import pandas as pd
import numpy as np

from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

import matplotlib.pyplot as plt

from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight
from tsfresh.feature_selection import select_features
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

from pyts.transformation import BOSS

from catboost import CatBoostClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
raw_prep_dir  = '../processed_data/'

In [81]:
X_q_data, y_data, id_list = load_q_data(raw_prep_dir + 'merged_dfq.csv')

In [83]:
id_list

0        1
1        2
2        3
3        4
4        5
      ... 
464    465
465    466
466    467
467    468
468    469
Name: id, Length: 469, dtype: int64

In [84]:
X_q_data.head()

,id,age,age_at_diagnosis,bmi,height,weight,gender,handedness,appearance_in_kinship,01,...,21,22,23,24,25,26,27,28,29,30
0,1,56,56,26.061679,173,78,male,right,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2,81,69,27.920213,193,104,male,right,False,True,...,True,True,True,False,True,False,True,False,True,False
2,3,45,45,26.989619,170,78,female,right,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,67,63,34.720883,161,90,female,right,False,False,...,True,True,True,False,False,True,True,True,False,False
4,5,75,65,29.069767,172,86,male,left,False,True,...,False,True,True,True,True,True,True,False,False,False


In [85]:
X = pd.read_csv("X_features_efficient.csv")

In [78]:
X['id'] = X_q_data[['id']].reset_index()['id']

In [87]:
X['id'] = X_q_data.id

In [88]:
X

,Unnamed: 0,feat_0__variance_larger_than_standard_deviation,feat_0__has_duplicate_max,feat_0__has_duplicate_min,feat_0__has_duplicate,feat_0__sum_values,feat_0__abs_energy,feat_0__mean_abs_change,feat_0__mean_change,feat_0__mean_second_derivative_central,...,feat_99__fourier_entropy__bins_10,feat_99__fourier_entropy__bins_100,feat_99__permutation_entropy__dimension_3__tau_1,feat_99__permutation_entropy__dimension_4__tau_1,feat_99__permutation_entropy__dimension_5__tau_1,feat_99__permutation_entropy__dimension_6__tau_1,feat_99__permutation_entropy__dimension_7__tau_1,feat_99__query_similarity_count__query_None__threshold_0.0,feat_99__mean_n_absolute_max__number_of_maxima_7,id
0,0,0.0,0.0,0.0,0.0,-0.237587,0.007123,0.001185,9.279604e-06,-4.920161e-08,...,0.200814,0.541904,1.192531,1.767280,2.368939,2.974586,3.555699,NaN,9.852452,1
1,1,0.0,0.0,0.0,0.0,0.301411,0.072681,0.003323,-7.561774e-06,2.239045e-06,...,0.294982,0.746059,1.281404,1.933244,2.624203,3.334902,4.027040,NaN,11.080031,2
2,2,0.0,0.0,0.0,0.0,0.350752,0.005038,0.001196,-1.071291e-05,-2.835534e-06,...,0.181214,0.561299,1.268612,1.928938,2.607994,3.264167,3.846058,NaN,9.485008,3
3,3,0.0,0.0,0.0,0.0,0.113223,0.004691,0.001471,-2.914391e-07,1.753255e-08,...,0.273489,0.974566,1.260238,1.907756,2.564959,3.215555,3.857449,NaN,5.818347,4
4,4,0.0,0.0,0.0,0.0,-0.082396,0.104001,0.002777,5.806350e-06,-3.081235e-06,...,0.170467,0.922702,1.257096,1.844994,2.455285,3.084139,3.707883,NaN,4.636343,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,464,0.0,0.0,0.0,0.0,-0.170825,0.009722,0.001286,5.856431e-06,8.644361e-07,...,0.228465,0.645639,1.256750,1.882662,2.532292,3.156781,3.727529,NaN,7.078482,465
465,465,0.0,0.0,0.0,0.0,-0.015599,0.004124,0.000948,2.717958e-06,-5.097756e-07,...,0.204871,0.557996,1.257459,1.890017,2.565472,3.263690,3.948355,NaN,5.221625,466
466,466,0.0,0.0,0.0,0.0,0.165183,0.244611,0.006510,2.862241e-05,-1.996930e-05,...,0.136002,0.686744,1.316578,1.991956,2.723216,3.460727,4.192557,NaN,5.742488,467
467,467,0.0,0.0,0.0,0.0,0.186751,0.012028,0.002237,-5.671895e-06,4.020430e-06,...,0.273489,0.756101,1.276855,1.931113,2.636961,3.375283,4.106112,NaN,7.137284,468


In [181]:
X_train_id, X_test_id, y_train, y_test = train_test_split_ids (id_list, y_data, test_size = 0.2, stratify = True)

In [183]:
X_train_id
print(X_train_id, y_train)

437    438
342    343
443    444
160    161
158    159
      ... 
275    276
421    422
182    183
68      69
49      50
Name: id, Length: 375, dtype: int64 437    2
342    1
443    0
160    2
158    1
      ..
275    2
421    1
182    2
68     1
49     2
Name: label, Length: 375, dtype: int64


In [151]:
X_train_time = X[X.id.isin(X_train_id)]
X_test_time = X[X.id.isin(X_test_id)]
X_train_q = X_q_data[X_q_data.id.isin(X_train_id)]
X_test_q = X_q_data[X_q_data.id.isin(X_test_id)]
print(X_train_time.to_numpy().shape, X_train_q.shape)

(375, 102566) (375, 39)


In [152]:
X_train_q

,id,age,age_at_diagnosis,bmi,height,weight,gender,handedness,appearance_in_kinship,01,...,21,22,23,24,25,26,27,28,29,30
0,1,56,56,26.061679,173,78,male,right,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2,81,69,27.920213,193,104,male,right,False,True,...,True,True,True,False,True,False,True,False,True,False
4,5,75,65,29.069767,172,86,male,left,False,True,...,False,True,True,True,True,True,True,False,False,False
5,6,72,60,39.328340,171,115,female,right,False,False,...,False,False,True,False,False,True,True,True,False,False
6,7,74,73,28.692653,181,94,male,right,False,True,...,True,False,True,False,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,463,65,65,27.777778,180,90,male,right,False,False,...,False,False,False,False,False,False,False,True,False,False
464,465,65,62,26.122449,175,80,male,right,True,True,...,False,False,False,False,True,False,False,False,False,False
465,466,84,84,25.013521,172,74,female,right,True,False,...,False,False,False,False,False,False,False,False,False,False
467,468,76,73,30.098969,198,118,male,right,False,False,...,True,False,False,False,True,True,True,True,False,False


In [153]:
prep = Preprocess_Q(feature_importance = True)
prep.fit(X_train_q)
X_q_train_prep = prep.transform(X_train_q)
X_q_test_prep = prep.transform(X_test_q)
pd.DataFrame(X_q_train_prep)

,0,1,2,3,4,5,6,7,8,9
0,-0.176471,-0.6250,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.588235,0.9375,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
2,0.352941,0.5625,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.058824,0.3750,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
4,0.823529,0.5000,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
370,0.352941,-0.0625,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
371,0.176471,-0.0625,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
372,1.470588,1.1250,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
373,0.823529,0.6250,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [154]:
X_train_features_clean = X_train_time.copy()
X_train_features_clean = X_train_features_clean.dropna(axis=1)
y_series = pd.Series(y_train, index=X_train_features_clean.index)
classes = np.unique(y_train)

selected_sets = []
for c in classes:
     y_bin = (y_series == c).astype(int)
     sel_c = select_features(X_train_features_clean, y_bin, fdr_level=0.0000001)
     selected_sets.append(set(sel_c.columns))
     selected_cols = sorted(set().union(*selected_sets))
X_train_selected = X_train_features_clean[selected_cols]
print(f"Selected {len(selected_cols)} / {X_train_features_clean.shape[1]} features")

Selected 118 / 102434 features


In [179]:
selected_cols

['feat_48__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',
 'feat_48__quantile__q_0.7',
 'feat_48__quantile__q_0.8',
 'feat_49__quantile__q_0.7',
 'feat_50__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',
 'feat_50__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',
 'feat_50__quantile__q_0.7',
 'feat_52__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',
 'feat_52__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',
 'feat_52__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',
 'feat_52__number_peaks__n_1',
 'feat_52__partial_autocorrelation__lag_2',
 'feat_52__permutation_entropy__dimension_3__tau_1',
 'feat_52__permutation_entropy__dimension_4__tau_1',
 'feat_52__permutation_entropy__dimension_5__tau_1',
 'feat_52__permutation_entropy__dimension_6__tau_1',
 'feat_52__permutation_entropy__dimension_7__tau_1',
 'feat_54__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',
 'feat_54__change_quantiles__f_agg_"mean"__is

In [158]:
X_test_selected = X_test_time[selected_cols]

In [177]:
r_scaler = RobustScaler()
X_train_prep = r_scaler.fit_transform(X_train_selected)
X_test_prep = r_scaler.transform(X_test_selected)

r_scaler = RobustScaler()
X_train_prep = X_train_selected
X_test_prep = X_test_selected

In [178]:
X_combined_train = np.concatenate((X_train_prep, X_q_train_prep), axis = 1)
X_combined_test = np.concatenate((X_test_prep, X_q_test_prep), axis = 1)
X_combined_train.shape

(375, 128)

In [174]:
sv_model_combined = SVC(kernel = 'rbf', class_weight = 'balanced', C = 1)
sv_model_combined.fit(X_combined_train, y_train)

,C,1
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,'balanced'
,verbose,False


In [175]:
y_pred_combo = sv_model_combined.predict(X_combined_test)

In [176]:
y_pred_combo = sv_model_combined.predict(X_combined_test)
balanced_accuracy_score(y_test, y_pred_combo)

0.3087121212121212

In [167]:
xgb_model_pca = XGBClassifier()

In [170]:
xgb = XGBClassifier()

sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)


xgb = XGBClassifier()

xgb.fit(X_combined_train, y_train, sample_weight=sample_weights)

y_pred_xgb = xgb.predict(X_combined_test)

In [171]:
xgb_accuracy = balanced_accuracy_score(y_test, y_pred_xgb)
f'xgb_accuracy: {xgb_accuracy}'

'xgb_accuracy: 0.3626152832674572'